# Setup and Imports

In [12]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
%cd /content/drive/MyDrive/Fall2021/538_Project

[Errno 2] No such file or directory: '/content/drive/MyDrive/Fall2021/538_Project'
/content


In [1]:
import pandas as pd

# Generation and evaluation of dataset

In [2]:
!pip install transformers
# !pip install sentencepiece
# !pip install streamlit
# !pip install --upgrade tensorflow
!pip install datasets
!pip install git+https://github.com/google-research/bleurt.git

     |████████████████████████████████| 3.1 MB 5.5 MB/s 
     |████████████████████████████████| 3.3 MB 37.9 MB/s 
     |████████████████████████████████| 596 kB 32.7 MB/s 
     |████████████████████████████████| 895 kB 38.4 MB/s 
     |████████████████████████████████| 61 kB 65 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 5.3 MB/s 
     |████████████████████████████████| 243 kB 39.1 MB/s 
     |████████████████████████████████| 1.1 MB 34.0 MB/s 
     |████████████████████████████████| 132 kB 46.4 MB/s 
     |████████████████████████████████| 160 kB 48.8 MB/s 
     |████████████████████████████████| 192 kB 45.9 MB/s 
     |████████████████████████████████| 271 kB 51.6 MB/s 
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-r66re8b3
  Running command git clone -q https://github.com/google-research/bleurt.

In [3]:
import torch
#CUDA_LAUNCH_BLOCKING=1
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Vamsi/T5_Paraphrase_Paws")  
model = AutoModelForSeq2SeqLM.from_pretrained("Vamsi/T5_Paraphrase_Paws")
model.to(device)

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [5]:
!wget https://cims.nyu.edu/~sbowman/multinli/multinli_1.0.zip

--2021-12-04 20:36:47--  https://cims.nyu.edu/~sbowman/multinli/multinli_1.0.zip
Resolving cims.nyu.edu (cims.nyu.edu)... 216.165.22.202
Connecting to cims.nyu.edu (cims.nyu.edu)|216.165.22.202|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 226850426 (216M) [application/zip]
Saving to: ‘multinli_1.0.zip’

multinli_1.0.zip    100%[===================>] 216.34M  23.8MB/s    in 9.2s    

2021-12-04 20:36:57 (23.5 MB/s) - ‘multinli_1.0.zip’ saved [226850426/226850426]



In [6]:
!unzip /content/multinli_1.0.zip

Archive:  /content/multinli_1.0.zip
   creating: multinli_1.0/
  inflating: multinli_1.0/multinli_1.0_dev_mismatched.jsonl  
  inflating: multinli_1.0/paper.pdf  
   creating: __MACOSX/
   creating: __MACOSX/multinli_1.0/
  inflating: __MACOSX/multinli_1.0/._paper.pdf  
  inflating: multinli_1.0/.DS_Store  
  inflating: __MACOSX/multinli_1.0/._.DS_Store  
  inflating: multinli_1.0/multinli_1.0_train.jsonl  
  inflating: multinli_1.0/multinli_1.0_dev_mismatched.txt  
 extracting: multinli_1.0/Icon       
  inflating: __MACOSX/multinli_1.0/._Icon  
  inflating: multinli_1.0/multinli_1.0_dev_matched.txt  
  inflating: multinli_1.0/multinli_1.0_dev_matched.jsonl  
  inflating: multinli_1.0/multinli_1.0_train.txt  
  inflating: multinli_1.0/README.txt  
  inflating: __MACOSX/multinli_1.0/._README.txt  
  inflating: __MACOSX/._multinli_1.0  


In [7]:
import datasets
metric = datasets.load_metric('bleurt')

Downloading:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

Using default BLEURT-Base checkpoint for sequence maximum length 128. You can use a bigger model for better results with e.g.: datasets.load_metric('bleurt', 'bleurt-large-512').


Downloading:   0%|          | 0.00/405M [00:00<?, ?B/s]

INFO:tensorflow:Reading checkpoint /root/.cache/huggingface/metrics/bleurt/default/downloads/extracted/887f2dc36c17f53c287f696681b8f7c947278407c1cf9f226662e16c8c0dc417/bleurt-base-128.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint bert_custom
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:bert_custom
INFO:tensorflow:... vocab_file:vocab.txt
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... do_lower_case:True
INFO:tensorflow:... max_seq_length:128
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating WordPiece tokenizer.
INFO:tensorflow:WordPiece tokenizer instantiated.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.
INFO:tensorflow:BLEURT initialized.


In [8]:

import string
def Jaccard_Similarity(s1, s2): 
    s1 = preprocess(s1)
    s2 = preprocess(s2)
    
    s1 = set(s1.lower().split()) 
    s2 = set(s2.lower().split())
    # print(s1)
    intersection = s1.intersection(s2)

    union = s2.union(s1)
        
    # Calculate Jaccard similarity score 
    # using length of intersection set divided by length of union set
    return float(len(intersection)) / len(union)    
    
def preprocess(a):
    table = str.maketrans(dict.fromkeys(string.punctuation))
    new_s = a.translate(table) 
    return new_s

# print(f'Jaccard Sim {Jaccard_Similarity(a, b)}')


In [68]:
def bestParaphrase( sentence, paraphrases ):
    # Add evaluation check to select the best paraphrase here and not select if
    # none of the paraphrases meet the requirements
    min_score = None
    best_paraphrase = None
    print(sentence)
    for paraphrase in paraphrases:
      if paraphrase == sentence:
        continue
      score = metric.compute(predictions=[paraphrase], references=[sentence])['scores'][0]
      # if max_score == None or score > max_score:
      #   if score > 0.4:
      #     if max_score == None or score > max_score:
      #       max_score = score
      #       best_paraphrase = paraphrase
      #   elif score > max_score and score > 0.4:
      #     max_score = score
      #     best_paraphrase = paraphrase
      # elif score > max_score:
      #   max_score = score
      #   best_paraphrase = paraphrase
      if score > 0.7:
        if Jaccard_Similarity(sentence, paraphrase) <= 0.75:
          print("Potential paraphrase:", score, paraphrase, Jaccard_Similarity(sentence, paraphrase))
          if min_score == None or Jaccard_Similarity(sentence, paraphrase) < min_score:
            min_score = Jaccard_Similarity(sentence, paraphrase)
            best_paraphrase = paraphrase
        else:
          print("Jaccard above 0.75:", score, paraphrase, Jaccard_Similarity(sentence, paraphrase))
      else:
        print("BLEURT below 0.7", score, paraphrase)
        # print(score, paraphrase)
    # print(type(max_score), max_score)
    print("Best paraphrase:", best_paraphrase, min_score)
    return best_paraphrase

In [69]:
def generateParaphrase( sentence ):
    text =  "paraphrase: " + sentence + " </s>"

    encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
    #input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding["attention_mask"].to("cuda")
    input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)

    outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      max_length=256,
      do_sample=True,
      top_k=120,
      top_p=0.95,
      early_stopping=True,
      num_return_sequences=10
    )
    paraphrases = []
    # print( "Sentence: ", sentence )
    for output in outputs:
      line = tokenizer.decode(output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      paraphrases.append( line )
      # print( "Para: ", line )
    return bestParaphrase( sentence, paraphrases )

In [70]:
df = pd.read_json( '/content/multinli_1.0/multinli_1.0_dev_mismatched.jsonl', lines=True )
# df = pd.read_csv('/content/drive/MyDrive/NLP/paraphrased_mismatched_dataset_v2.csv')

In [71]:
df

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,"[contradiction, contradiction, contradiction, ...",letters,contradiction,75290c,75290,Your contribution helped make it possible for ...,( ( Your contribution ) ( ( helped ( make ( it...,(ROOT (S (NP (PRP$ Your) (NN contribution)) (V...,Your contributions were of no help with our st...,( ( Your contributions ) ( ( were ( of ( ( no ...,(ROOT (S (NP (PRP$ Your) (NNS contributions)) ...
1,"[contradiction, contradiction, contradiction, ...",verbatim,contradiction,133794c,133794,"The answer has nothing to do with their cause,...",( ( ( ( ( ( The answer ) ( ( ( ( has nothing )...,(ROOT (S (S (NP (DT The) (NN answer)) (VP (VBZ...,Dictionaries are indeed exercises in bi-unique...,( Dictionaries ( ( ( are indeed ) ( exercises ...,(ROOT (S (NP (NNS Dictionaries)) (VP (VBP are)...
2,"[contradiction, neutral, entailment, entailmen...",verbatim,entailment,3628c,3628,We serve a classic Tuscan meal that includes ...,( We ( ( serve ( ( a ( classic ( Tuscan meal )...,(ROOT (S (NP (PRP We)) (VP (VBP serve) (NP (NP...,We serve a meal of Florentine terrine.,( We ( ( serve ( ( a meal ) ( of ( Florentine ...,(ROOT (S (NP (PRP We)) (VP (VBP serve) (NP (NP...
3,"[contradiction, contradiction, contradiction, ...",letters,contradiction,89411c,89411,"A few months ago, Carl Newton and I wrote a le...","( ( ( A ( few months ) ) ago ) ( , ( ( ( ( Car...",(ROOT (S (ADVP (NP (DT A) (JJ few) (NNS months...,Carl Newton and I have never had any other pre...,( ( ( ( Carl Newton ) and ) I ) ( ( ( have nev...,(ROOT (S (NP (NP (NNP Carl) (NNP Newton)) (CC ...
4,"[entailment, entailment, entailment, entailmen...",facetoface,entailment,136158e,136158,"I was on this earth you know, I've lived on th...",( I ( ( was ( on ( ( this earth ) ( you ( know...,(ROOT (S (NP (PRP I)) (VP (VBD was) (PP (IN on...,I don't yet know the reason why I have lived o...,( I ( ( ( ( do n't ) yet ) ( ( know ( the reas...,(ROOT (S (NP (PRP I)) (VP (VBP do) (RB n't) (A...
...,...,...,...,...,...,...,...,...,...,...,...
9995,"[contradiction, contradiction, neutral, entail...",facetoface,contradiction,47686c,47686,Do you watch that?,( ( ( Do you ) ( watch that ) ) ? ),(ROOT (SQ (VBP Do) (NP (PRP you)) (VP (VB watc...,Can you see?,( ( ( Can you ) see ) ? ),(ROOT (SQ (MD Can) (NP (PRP you)) (VP (VB see)...
9996,"[contradiction, contradiction, contradiction, ...",verbatim,contradiction,126829c,126829,"To a Western ear, the most predictable of lang...","( ( ( ( ( To ( ( ( a ( Western ear ) ) , ) ( (...",(ROOT (SINV (S (S (PP (TO To) (NP (NP (DT a) (...,"To the Western ear, the least predictable of l...","( ( To ( the ( Western ear ) ) ) ( , ( ( ( the...",(ROOT (S (PP (TO To) (NP (DT the) (JJ Western)...
9997,"[contradiction, contradiction, contradiction, ...",nineeleven,contradiction,37368c,37368,The recorder captured the sounds of loud thump...,( ( The recorder ) ( ( ( ( captured ( ( the so...,(ROOT (S (NP (DT The) (NN recorder)) (VP (VP (...,The recorder didn't capture any of the sounds.,( ( The recorder ) ( ( ( did n't ) ( capture (...,(ROOT (S (NP (DT The) (NN recorder)) (VP (VBD ...
9998,"[neutral, neutral, entailment, neutral, neutral]",facetoface,neutral,97460n,97460,That's a good attitude!,( That ( ( 's ( a ( good attitude ) ) ) ! ) ),(ROOT (S (NP (DT That)) (VP (VBZ 's) (NP (DT a...,"You feel good about this, don't you?","( ( You ( ( feel good ) ( about this ) ) ) ( ,...",(ROOT (SINV (S (NP (PRP You)) (VP (VBP feel) (...


In [72]:
df[ [ 'sentence1', 'sentence2', 'gold_label' ] ].head()

,sentence1,sentence2,gold_label
0,Your contribution helped make it possible for ...,Your contributions were of no help with our st...,contradiction
1,"The answer has nothing to do with their cause,...",Dictionaries are indeed exercises in bi-unique...,contradiction
2,We serve a classic Tuscan meal that includes ...,We serve a meal of Florentine terrine.,entailment
3,"A few months ago, Carl Newton and I wrote a le...",Carl Newton and I have never had any other pre...,contradiction
4,"I was on this earth you know, I've lived on th...",I don't yet know the reason why I have lived o...,entailment


In [73]:
from tqdm import tqdm
tqdm.pandas()

In [74]:
df = df.sample(n=5, random_state=42)
df['sentence1dash'] = df['sentence1'].apply( generateParaphrase )

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


My dad still doesn't believe me.
Potential paraphrase: 0.8298517465591431 My dad still does not believe in me. 0.5555555555555556
BLEURT below 0.7 0.6830822825431824 My dad doesn't believe me yet.
Jaccard above 0.75: 0.8778197169303894 My dad still doesn't believe in me. 0.8571428571428571
Potential paraphrase: 0.9573001265525818 My father still doesn't believe me. 0.7142857142857143
Best paraphrase: My dad still does not believe in me. 0.5555555555555556
Well, because it was really interesting stuff.
BLEURT below 0.7 0.6721789240837097 Also, because it was really interesting stuff.
BLEURT below 0.7 0.27884548902511597 It was really interesting stuff, well...
Jaccard above 0.75: 0.8371367454528809 Because it was really interesting stuff. 0.8571428571428571
BLEURT below 0.7 0.6516732573509216 Actually, it was really interesting stuff.
BLEURT below 0.7 0.4208700358867645 We thought it was really interesting stuff.
Best paraphrase: None None
People generally seem to find dialect study int

In [ ]:
# df.to_csv("/content/drive/MyDrive/NLP/paraphrased_mismatched_dataset_v2.csv")

In [16]:
df['sentence2dash'] = df['sentence2'].progress_apply( generateParaphrase )

  0%|          | 0/10000 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
100%|██████████| 10000/10000 [2:18:34<00:00,  1.20it/s]


In [17]:
# df.to_csv("/content/drive/MyDrive/NLP/paraphrased_mismatched_dataset_v2.csv")

# Checking accuracy

In [ ]:
datasetDf = pd.read_csv( 'paraphrased_dataset.csv' )

In [ ]:
datasetDf.head(2)

,Unnamed: 0,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse,sentence1dash,sentence2dash
0,6252,"['contradiction', 'contradiction', 'contradict...",fiction,contradiction,44271c,44271,He wore a simple leather breastplate with a si...,( He ( ( ( ( wore ( a ( simple ( leather breas...,(ROOT (S (NP (PRP He)) (VP (VBD wore) (NP (DT ...,He was wearing complex satin breastplate that ...,( He ( ( was ( wearing ( ( complex ( satin bre...,(ROOT (S (NP (PRP He)) (VP (VBD was) (VP (VBG ...,"He wore a simple leather breastplate, with a s...",He was wearing a complex satin breastplate tha...
1,4684,"['contradiction', 'contradiction', 'contradict...",fiction,contradiction,88994c,88994,"It will be COLOSSAL!""",( It ( ( ( will ( be COLOSSAL ) ) ! ) '' ) ),(ROOT (S (NP (PRP It)) (VP (MD will) (VP (VB b...,It will be miniscule.,( It ( ( will ( be miniscule ) ) . ) ),(ROOT (S (NP (PRP It)) (VP (MD will) (VP (VB b...,"It is COLOSSAL!""",It will be Miniscule.


In [ ]:
!python pytorch-nli/scripts/download_pretrained.py -m bilstm -d snli

1491it [00:01, 970.17it/s]
 [*] model best-bilstm-snli-params.pt downloaded from the google drive.


In [ ]:
!python pytorch-nli/scripts/download_pretrained.py -m bilstm -d multinli

3010it [00:02, 1480.23it/s]
 [*] model best-bilstm-multinli-params.pt downloaded from the google drive.


In [ ]:
!pip install torchtext==0.4.0

     |████████████████████████████████| 53 kB 1.2 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


This evaluator takes the mnli file multinli_1.0_dev_mismatched.jsonl as the test dataset. A hack would be to just replace this file with out own dataset and run the evaluator. The below method is from the built in function of TORCHTEXT.DATASETS.NLI.

def splits(cls, text_field, label_field, parse_field=None, genre_field=None,
               root='.data',
               train='multinli_1.0_train.jsonl',
               validation='multinli_1.0_dev_matched.jsonl',
             test='multinli_1.0_dev_mismatched.jsonl'):

In [ ]:
!python pytorch-nli/evaluate.py -m bilstm -d multinli

program execution start: 2021-11-14 01:09:30.695728
preparing the dataset for training...
> /content/drive/MyDrive/Fall2021/538_Project/pytorch-nli/datasets/multinli.py(29)__init__()
-> self.TEXT.build_vocab(self.train, self.dev)
(Pdb) c


Traceback (most recent call last):
  File "pytorch-nli/evaluate.py", line 99, in <module>
    task = Evaluate()
  File "pytorch-nli/evaluate.py", line 27, in __init__
    self.validation_accuracy, self.model_options, model_dict = self.load_model()
  File "pytorch-nli/evaluate.py", line 37, in load_model
    model = torch.load('{}/{}/{}/best-{}-{}-params.pt'.format(self.args.results_dir, self.args.model, self.args.dataset, self.args.model, self.args.dataset))
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 594, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 230, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr

In [ ]:
!python pytorch-nli/evaluate.py -m bilstm -d multinli

program execution start: 2021-11-14 02:17:56.414954
preparing the dataset for training...
> /content/drive/My Drive/Fall2021/538_Project/pytorch-nli/datasets/multinli.py(29)__init__()
-> self.TEXT.build_vocab(self.train, self.dev)
(Pdb) c
Traceback (most recent call last):
  File "pytorch-nli/evaluate.py", line 99, in <module>
    task = Evaluate()
  File "pytorch-nli/evaluate.py", line 30, in __init__
    self.model.load_state_dict(model_dict)
  File "/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py", line 1483, in load_state_dict
    self.__class__.__name__, "\n\t".join(error_msgs)))
RuntimeError: Error(s) in loading state_dict for BiLSTM:
	size mismatch for embedding.weight: copying a param with shape torch.Size([75412, 300]) from checkpoint, the shape in current model is torch.Size([75323, 300]).


In [ ]:
!ls .data/multinli/multinli_1.0/

multinli_1.0_dev_matched.jsonl	   multinli_1.0_dev_mismatched.txt  paper.pdf
multinli_1.0_dev_matched.txt	   multinli_1.0_train.jsonl	    README.txt
multinli_1.0_dev_mismatched.jsonl  multinli_1.0_train.txt
